In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import umap.umap_ as umap
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import os
import sys

In [ ]:
spectra_data_path = "/Users/max/astroTest/data_analysis/data/kf2/fos_spectra_post/"
spectra_data = []

def erg_to_jansky(flux_density_erg):
    # Convert flux density to jansky
    flux_density_erg = np.asarray(flux_density_erg)
    flux_density_jy = (flux_density_erg * (u.erg / (u.s * u.cm**2 * u.Angstrom))).to(u.Jy, equivalencies=u.spectral_density(1 * u.Angstrom))
    return flux_density_jy
  

for source in os.listdir(spectra_data_path):
    file_path = "data/kf2/fos_spectra_post/" + source 
    df = pd.read_csv(file_path, sep='\s+', header=None)
    df.columns = ["Wavelength", "Flux", "Error", "(Possibly) Data Quality Flag"]
    df = df.drop("(Possibly) Data Quality Flag", axis=1)

    # Flipping spectra so it goes from shortest wavelength to longest (Most already do, some don't)
    if (df.iat[-1, 0]-df.iat[0, 0]) < 0:
        df = df[::-1]

    spectra_data.append(df)

In [ ]:
from scipy.interpolate import interp1d

def fourier_resample_logscale(wavelength, flux, new_wavelength):
    # Transform to frequency domain using FFT
    frequency = np.fft.fftfreq(len(wavelength), np.mean(np.diff(np.log10(wavelength))))
    spectrum_fft = np.fft.fft(flux)
    
    # Interpolate in the frequency domain
    interpolator = interp1d(frequency, spectrum_fft, kind='linear', fill_value="extrapolate")
    new_spectrum_fft = interpolator(np.fft.fftfreq(len(new_wavelength), np.mean(np.diff(np.log10(new_wavelength)))))

    # Transform back to wavelength domain using IFFT
    new_flux = np.fft.ifft(new_spectrum_fft)
    
    
    resampled_spectrum = np.array(new_flux.real)
    
    return resampled_spectrum

In [ ]:
adjusted_spectra = []

# Determined using distribution of spectra start/stop intervals 
fft_start = 1590 
fft_end = fft_start + 2000 
fft_density = 1000
new_wavelength = np.logspace(np.log10(fft_start), np.log10(fft_end), fft_density)


for spectrum in spectra_data:
    fft_sepctra = fourier_resample_logscale(wavelength=spectrum["Wavelength"], flux=spectrum["Flux"], new_wavelength=new_wavelength)
    adjusted_spectra.append(fft_sepctra)


spectra = pd.DataFrame(adjusted_spectra, columns=[new_wavelength])

In [ ]:
X = spectra
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
pca = PCA(n_components=2)
principal_components = pca.fit_transform(X_scaled)
components = pca.components_  # Principal components
explained_variance_ratio = pca.explained_variance_ratio_  # Explained variance ratio
plt.scatter(principal_components[:, 0], principal_components[:, 1])
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('PCA - Spectra Data')
plt.show()

In [ ]:
umap_model = umap.UMAP(n_components=2)
from sklearn.cluster import KMeans

# Fit the UMAP model to the data
umap_result = umap_model.fit_transform(spectra)

# Access the UMAP embeddings
umap_embeddings = umap_result

# Plot the UMAP embeddings
plt.scatter(umap_embeddings[:, 0], umap_embeddings[:, 1])
plt.xlabel('UMAP Component 1')
plt.ylabel('UMAP Component 2')
plt.title('UMAP Embeddings')
plt.show()

'''
kmeans = KMeans(n_clusters=3, random_state=42)
cluster_labels = kmeans.fit_predict(umap_embeddings)

# Plot the UMAP embeddings with cluster colors
plt.scatter(umap_embeddings[:, 0], umap_embeddings[:, 1], c=cluster_labels)
plt.xlabel('UMAP Component 1')
plt.ylabel('UMAP Component 2')
plt.title('UMAP Clustering')
plt.colorbar(label='Cluster')
plt.show()
'''
